In [1]:
import pandas as pd
import re

In [17]:

df = pd.read_excel('/Users/thor/Documents/EC_utbildning/R/regression_analysis_R/data_raw/Skrapning_v3_ForetagNoleasing.xlsx')

In [18]:
df.columns

Index(['web-scraper-order', 'web-scraper-start-url', 'link', 'Company',
       'Location', 'Year', 'Fuel', 'Miles', 'Gear', 'Price', 'Date', 'Href',
       'Href-href', 'Name'],
      dtype='object')

In [19]:
df = df[['Company','Location', 'Year', 'Fuel', 'Miles', 'Gear', 'Price', 'Name']]

In [20]:
df

,Company,Location,Year,Fuel,Miles,Gear,Price,Name
0,Företag,Stockholm,2021,Bensin,3 089 mil,Manuell,186 900 kr,Seat León Sportstourer 1.5 TSI 130 HK STYLE
1,Företag,Stockholm,2021,Miljöbränsle/Hybrid,8 057 mil,Automat,329 900 kr,Volvo V60 Recharge T6 Inscription Expression (...
2,Företag,Stockholm,2023,El,2 045 mil,Automat,419 900 kr,Volvo C40 Recharge Single Motor Core (SELEKT)
3,NaN,Östergötland,2018,Bensin,14 854 mil,Automat,189 900 kr,Mitsubishi Outlander 2.0 4WD 7-sits V-Däck ing...
4,NaN,Örebro,2008,Bensin,30 998 mil,Manuell,39 900 kr,"Volvo V70 2.4 Classic, Momentum Euro 4"
...,...,...,...,...,...,...,...,...
12489,NaN,Stockholm,1988,Bensin,7 500 mil,Automat,69 900 kr,Volvo 340 3DR 1.4 VARIOMATIC TOPPSKICK LÅGMIL
12490,NaN,Uppsala,2021,El,147 mil,Automat,349 000 kr,Opel MOVANO e L3H2 70kWh AUT Business NAV Enda...
12491,NaN,Skåne,2011,Miljöbränsle/Hybrid,15 900 mil,Automat,109 900 kr,Toyota Auris Hybrid e-CVT 136hk
12492,NaN,Örebro,2010,Bensin,17 832 mil,Manuell,32 900 kr,Nissan Micra 5-dörrar 1.2 Euro 4


In [21]:
def clean_miles_price(df):
    # Remove the ' mil' and ' kr' substrings, and replace spaces
    df['Miles'] = df['Miles'].str.replace(' mil', '').str.replace(' ', '')
    df['Price'] = df['Price'].str.replace(' kr', '').str.replace(' ', '')

    df['Miles'] = pd.to_numeric(df['Miles'], errors='coerce')
    df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
    df.dropna(subset=['Miles', 'Price'], inplace=True)
    
    df['Miles'] = df['Miles'].astype(int)
    df['Price'] = df['Price'].astype(int)
    
    return df

print(df.shape)
df = clean_miles_price(df)
print(df.shape)

(12494, 8)
(11598, 8)


In [22]:
df

,Company,Location,Year,Fuel,Miles,Gear,Price,Name
0,Företag,Stockholm,2021,Bensin,3089,Manuell,186900,Seat León Sportstourer 1.5 TSI 130 HK STYLE
1,Företag,Stockholm,2021,Miljöbränsle/Hybrid,8057,Automat,329900,Volvo V60 Recharge T6 Inscription Expression (...
2,Företag,Stockholm,2023,El,2045,Automat,419900,Volvo C40 Recharge Single Motor Core (SELEKT)
3,NaN,Östergötland,2018,Bensin,14854,Automat,189900,Mitsubishi Outlander 2.0 4WD 7-sits V-Däck ing...
4,NaN,Örebro,2008,Bensin,30998,Manuell,39900,"Volvo V70 2.4 Classic, Momentum Euro 4"
...,...,...,...,...,...,...,...,...
12489,NaN,Stockholm,1988,Bensin,7500,Automat,69900,Volvo 340 3DR 1.4 VARIOMATIC TOPPSKICK LÅGMIL
12490,NaN,Uppsala,2021,El,147,Automat,349000,Opel MOVANO e L3H2 70kWh AUT Business NAV Enda...
12491,NaN,Skåne,2011,Miljöbränsle/Hybrid,15900,Automat,109900,Toyota Auris Hybrid e-CVT 136hk
12492,NaN,Örebro,2010,Bensin,17832,Manuell,32900,Nissan Micra 5-dörrar 1.2 Euro 4


In [23]:
# Define the function that extracts information from a single vehicle description
def extract_vehicle_info(description):
    # Regex patterns
    brand_model_pattern = r'^(\w+[\-\w]*\b) (\w+[\-\w]*\b(?: \w+[\-\w]*\b)?)'
    engine_volume_pattern = r'(\d+\.\d+|\d+)[\s\-](TSI|TDI|D[\-\s]4D|AWD|DIG-T/MHEV|e)'
    horsepower_pattern = r'(\d+)\s?(HK|hp|hk)'
    
    # Extract brand and model
    brand_model_match = re.search(brand_model_pattern, description)
    brand = brand_model_match.group(1) if brand_model_match else 'Unknown'
    model = brand_model_match.group(2) if brand_model_match else 'Unknown'
    
    # Extract engine volume
    engine_volume_match = re.search(engine_volume_pattern, description)
    engine_volume = engine_volume_match.group(1) if engine_volume_match else 'Unknown'
    
    # Extract horsepower
    horsepower_match = re.search(horsepower_pattern, description)
    horsepower = horsepower_match.group(1) if horsepower_match else 'Unknown'
    
    return brand, model, engine_volume, horsepower

# Apply the function to the 'Description' column and expand the results into new columns
df[['Brand', 'Model', 'Engine Volume', 'Horsepower']] = df.apply(lambda row: pd.Series(extract_vehicle_info(row['Name'])), axis=1)


In [24]:
df

,Company,Location,Year,Fuel,Miles,Gear,Price,Name,Brand,Model,Engine Volume,Horsepower
0,Företag,Stockholm,2021,Bensin,3089,Manuell,186900,Seat León Sportstourer 1.5 TSI 130 HK STYLE,Seat,León Sportstourer,1.5,130
1,Företag,Stockholm,2021,Miljöbränsle/Hybrid,8057,Automat,329900,Volvo V60 Recharge T6 Inscription Expression (...,Volvo,V60 Recharge,Unknown,Unknown
2,Företag,Stockholm,2023,El,2045,Automat,419900,Volvo C40 Recharge Single Motor Core (SELEKT),Volvo,C40 Recharge,Unknown,Unknown
3,NaN,Östergötland,2018,Bensin,14854,Automat,189900,Mitsubishi Outlander 2.0 4WD 7-sits V-Däck ing...,Mitsubishi,Outlander 2,Unknown,Unknown
4,NaN,Örebro,2008,Bensin,30998,Manuell,39900,"Volvo V70 2.4 Classic, Momentum Euro 4",Volvo,V70 2,Unknown,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...
12489,NaN,Stockholm,1988,Bensin,7500,Automat,69900,Volvo 340 3DR 1.4 VARIOMATIC TOPPSKICK LÅGMIL,Volvo,340 3DR,Unknown,Unknown
12490,NaN,Uppsala,2021,El,147,Automat,349000,Opel MOVANO e L3H2 70kWh AUT Business NAV Enda...,Opel,MOVANO e,Unknown,Unknown
12491,NaN,Skåne,2011,Miljöbränsle/Hybrid,15900,Automat,109900,Toyota Auris Hybrid e-CVT 136hk,Toyota,Auris Hybrid,Unknown,136
12492,NaN,Örebro,2010,Bensin,17832,Manuell,32900,Nissan Micra 5-dörrar 1.2 Euro 4,Nissan,Micra 5-dörrar,Unknown,Unknown


In [25]:
print(df.shape)
df = df.drop_duplicates()
print(df.shape)

(11598, 12)
(11525, 12)


In [26]:
df.describe(include='all')

,Company,Location,Year,Fuel,Miles,Gear,Price,Name,Brand,Model,Engine Volume,Horsepower
count,115,11525,11525.000000,11525,11525.000000,11513,1.152500e+04,11525,11525,11525,11525,11525
unique,1,23,NaN,4,NaN,2,NaN,11142,66,2643,52,264
top,Företag,Stockholm,NaN,Diesel,NaN,Automat,NaN,Volvo XC40 T2 FWD Momentum,Volkswagen,Model 3,Unknown,Unknown
freq,115,3531,NaN,4551,NaN,8732,NaN,15,1531,115,9170,6416
mean,NaN,NaN,2017.471236,NaN,9557.910803,NaN,2.746411e+05,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,5.606110,NaN,6684.797163,NaN,2.260878e+05,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,1930.000000,NaN,1.000000,NaN,1.000000e+00,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,2016.000000,NaN,4356.000000,NaN,1.499000e+05,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,2018.000000,NaN,8575.000000,NaN,2.298000e+05,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,2021.000000,NaN,13667.000000,NaN,3.390000e+05,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.replace()

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11525 entries, 0 to 12493
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Company        115 non-null    object
 1   Location       11525 non-null  object
 2   Year           11525 non-null  int64 
 3   Fuel           11525 non-null  object
 4   Miles          11525 non-null  int64 
 5   Gear           11513 non-null  object
 6   Price          11525 non-null  int64 
 7   Name           11525 non-null  object
 8   Brand          11525 non-null  object
 9   Model          11525 non-null  object
 10  Engine Volume  11525 non-null  object
 11  Horsepower     11525 non-null  object
dtypes: int64(3), object(9)
memory usage: 1.1+ MB


In [31]:
df.to_csv('/Users/thor/Documents/EC_utbildning/R/regression_analysis_R/data_preprocessed/skrapning_v3_cleaned.csv')